In [1]:
import pandas as pd
import json

tds_df = pd.read_excel("final_scores.xlsx")
tds_df.head(4)

,username,likes_given,likes_recieved,created_a_new_topic,replied,user's_post_was_quoted,user_marked_a_post_as_solution,Solutions_posted,initial_score,z_score
0,Raunak_Sen,10,0,0,3,0,0,0,7.1,0.040926
1,Mayur,6,0,1,2,0,0,0,5.4,-0.078642
2,23f1001468,0,1,1,2,0,0,0,3.1,-0.240411
3,gNIKHIL,6,7,1,11,0,0,0,16.6,0.709103


In [2]:
tds_df["user_id"] = None # Creating a new column in the dataframe

In [3]:
# load the data_dict of TDS user_actions json file
with open('data_dict_TDS_T12024.json') as f:
    data_dict = json.load(f)

for user in data_dict.keys():
    try:
        user_id = data_dict[user][0]["user_id"]
        # print(user_id)
        # Assign the user_id to the tds_df where username matches with user
        tds_df.loc[tds_df['username'] == user, 'user_id'] = user_id
    except Exception as e:
        print(f"{user} : {e}")

tds_df.head()


Rajeev_kumar_bhagat : list index out of range
harshil : list index out of range
22f3001166 : list index out of range
jkmadathil : list index out of range


,username,likes_given,likes_recieved,created_a_new_topic,replied,user's_post_was_quoted,user_marked_a_post_as_solution,Solutions_posted,initial_score,z_score,user_id
0,Raunak_Sen,10,0,0,3,0,0,0,7.1,0.040926,11915
1,Mayur,6,0,1,2,0,0,0,5.4,-0.078642,7974
2,23f1001468,0,1,1,2,0,0,0,3.1,-0.240411,3009
3,gNIKHIL,6,7,1,11,0,0,0,16.6,0.709103,8189
4,SunnyArya01,0,1,1,0,0,0,0,1.7,-0.338880,4961


In [ ]:
columns = tds_df.columns
tds_df = tds_df[["username","user_id"]+list(columns[1:-1])] # changing the order of columns
tds_df.head()

,username,user_id,likes_given,likes_recieved,created_a_new_topic,replied,user's_post_was_quoted,user_marked_a_post_as_solution,Solutions_posted,initial_score,z_score
0,Raunak_Sen,11915,10,0,0,3,0,0,0,7.1,0.040926
1,Mayur,7974,6,0,1,2,0,0,0,5.4,-0.078642
2,23f1001468,3009,0,1,1,2,0,0,0,3.1,-0.240411
3,gNIKHIL,8189,6,7,1,11,0,0,0,16.6,0.709103
4,SunnyArya01,4961,0,1,1,0,0,0,0,1.7,-0.338880


In [5]:
tds_df[tds_df["z_score"]>1.96]

,username,user_id,likes_given,likes_recieved,created_a_new_topic,replied,user's_post_was_quoted,user_marked_a_post_as_solution,Solutions_posted,initial_score,z_score
75,Amit1,14354,40,43,6,141,0,12,13,170.2,11.512470
76,Learner_HS,13371,1,22,7,23,0,2,2,41.4,2.453397
152,23ds1000133,12082,9,18,11,16,0,0,0,39.3,2.305695
178,PremKumarGupta,15418,1,34,2,87,1,4,7,95.6,6.265523
180,Nithish_Kumar,4961,36,11,0,69,0,5,7,82.0,5.308974
198,GKay,1644,6,39,3,47,0,4,4,71.0,4.535296
267,23f2000690,24630,2,34,7,9,0,0,0,38.1,2.221293


In [15]:
with open('discourse-analytics-user-metrics-per-date@discourse-onlinedegree-iitm-ac-in-2025-01-31.dcqresult.json') as f:
    data_dict = json.load(f)
    columns = data_dict["columns"]
all_users_data = pd.DataFrame(columns=columns)
all_users_data

,user_id,likes_received,likes_given,topics_viewed,posts_read,days_visited,solutions,cheers


In [16]:
for row in data_dict["rows"]: # 41418 rows in total
    all_users_data.loc[len(all_users_data)] = row # APPEND THE ROW TO THE DATAFRAME

all_users_data.sample(4)

,user_id,likes_received,likes_given,topics_viewed,posts_read,days_visited,solutions,cheers
4135,5176,0,1,0,0,0,0,22
34254,37995,0,0,0,0,0,0,0
11751,15294,0,0,0,0,0,0,0
15312,18907,0,0,0,0,0,0,3


In [17]:
print(len(all_users_data))

41418


In [19]:
weights_dict = { 'likes_given': 0.5, # 0.3
                "likes_received": 0.7,
                "created_a_new_topic": 1,
                "replied": 0.7,
                "user's_post_was_quoted": 0.6,
                "user_marked_a_post_as_solution":0.2,
                "Solutions_posted": 1
}
columns_to_be_ignored = ["initial_score",'subject','username', "user_id", 'overall_topics_count_of_this_subject', 'normalised_score', 'z_score'] # this was directly taken from previous notebook. Some column names might seem irrelevant. Please ignore them.

all_users_data["initial_score"] = sum(all_users_data[column]*weights_dict[column] for column in all_users_data.columns if column not in columns_to_be_ignored) # Initial score = sum(column_value*weight)

# merged_df["normalised_score"] = merged_df["initial_score"] / merged_df ["overall_topics_count_of_this_subject"] # normalised_score # THIS PART IS OMITTED AS WE ARE CALCULATING THE SCORES ONLY FOR TDS USERS AND NOT COMPARING THEM WITH OTHER SUBJECTS

all_users_data["z_score"] = round((all_users_data["initial_score"] - all_users_data["initial_score"].mean()) / all_users_data["initial_score"].std(),2) # z_score

final_scores = all_users_data[["username", "initial_score", "z_score"]]

KeyError: 'topics_viewed'